In [ ]:
import jax.numpy as jnp
import jax
from jax import experimental
from jax.experimental import optimizers
from jax import grad, jit, vmap
from jax import random
from scipy.special import jv
import os
import datetime
import time
import sys

import numpy as np
import pandas as pd
import time
import math
from jax import device_put

# Manuscript: "Target Acoustic Field and Transducer State Optimization using Diff-PAT"
# Authors: Tatsuki Fushimi, Kenta Yamamoto, Yoichi Ochiai
# Corresponding email: tfushimi@slis.tsukuba.ac.jp
# JAX version 0.2.17

In [ ]:
print(jax.__version__)

0.2.17


In [ ]:
import os
if not os.path.exists('results'):
    os.makedirs('results')

In [ ]:
N=16
pitch=0.0105
size=[N,N]
tra_x = np.array([ (i % N - (N//2-0.5))*pitch for i in range(N * N)]).reshape(N, N)
tra_y = tra_x.T
tra_z = np.full((N,N), 0.0)
P0=1.0
l_ambda = 346.0 / 40000.0
k = 2.0*math.pi/l_ambda
r0 = 0.005
dropping_threshold = 1000;

if False:
  np.savetxt('transducer_x.csv', np.array( [tra_x.T.ravel()] ).ravel(), delimiter=',')
  np.savetxt('transducer_y.csv', np.array( [tra_y.T.ravel()] ).ravel(), delimiter=',')
  np.savetxt('transducer_z.csv', np.array( [tra_z.T.ravel()] ).ravel(), delimiter=',')

In [ ]:
# Pre-calculating propagation parts
def prop_matrix(array_x, array_y, array_z, x_p, y_p, z_p, P0, k, r0):
  prop = []
  for xi in range(point_num):
    dist_map = jnp.sqrt( jnp.power((array_x - x_p[xi]), 2) + jnp.power((array_y - y_p[xi]), 2) + jnp.power((array_z - z_p[xi]), 2) )

    sin_alpha_map = jnp.sqrt( (jnp.power((array_x - x_p[xi]), 2) + jnp.power((array_y - y_p[xi]), 2)) ) / dist_map
    sin_alpha_map = jnp.where(sin_alpha_map == 0, jax.numpy.finfo(jnp.float64).tiny, sin_alpha_map)

    amplitude_map = ( 2 * jv(1, k*r0*sin_alpha_map) * P0 / (k*r0*sin_alpha_map*dist_map) )
    prop.append( jax.lax.complex( amplitude_map * jnp.cos(k*dist_map), amplitude_map * jnp.sin(k*dist_map) ) )
  
  prop = device_put(jnp.array(prop))

  return prop

In [ ]:
# Three types of propagation mode, A - Phase, B - Amplitude, C - Phase and Amplitude
@jit
def Propagation_A(Tr):
  part_A = Tr[0]
  trans_amplitude = jax.lax.complex(jnp.ones(size), jnp.zeros(size))
  trans_phase_copmlex = jax.lax.complex(jnp.cos(part_A), jnp.sin(part_A))
  return trans_amplitude, trans_phase_copmlex

@jit
def Propagation_C(Tr):
  part_A = Tr[0]
  part_B = Tr[1]

  tr_set = jax.lax.complex(part_A, part_B)
  temp_amp = jnp.abs(tr_set);
  temp_amp = temp_amp / jnp.max(temp_amp)
    
  trans_amplitude = jax.lax.complex(temp_amp, jnp.zeros(size))
  trans_phase = jnp.angle(tr_set)
  trans_phase_copmlex = jax.lax.complex(jnp.cos(trans_phase), jnp.sin(trans_phase))
  return trans_amplitude, trans_phase_copmlex

# Calculate Acoustic Pressure Field at the target point, output amplitude and phase. 
@jit 
def calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i):
  point_Re = jnp.sum(jnp.real(trans_phase_copmlex) * jnp.real(trans_amplitude*transducer_prop[i]) - jnp.imag(trans_phase_copmlex) * jnp.imag(trans_amplitude*transducer_prop[i]))
  point_Im = jnp.sum(jnp.real(trans_phase_copmlex) * jnp.imag(trans_amplitude*transducer_prop[i]) + jnp.imag(trans_phase_copmlex) * jnp.real(trans_amplitude*transducer_prop[i]))
  point_amp = jnp.sqrt( point_Re ** 2 + point_Im ** 2 )
  point_phase = jnp.angle(jax.lax.complex(point_Re, point_Im))
  return point_amp, point_phase

# Loss function depending on the target point state. i - Phase, ii - Amplitude, iii - Phase and Amplitude
@jit
def ErrFunc_ii(point_amp, target_amp):
  err = (point_amp - target_amp)**2
  return err

In [ ]:
@jit
def loss_func_Aii(Tr, transducer_prop, target_amp):
    trans_amplitude, trans_phase_copmlex = Propagation_A(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)
      err = ErrFunc_ii(point_amp, target_amp)
      loss += (1/point_num)*(err)
    return loss

@jit
def step_Aii(step, opt_state, transducer_prop, target_amp):
  value, grads = jax.value_and_grad(loss_func_Aii)(get_params(opt_state), transducer_prop, target_amp)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Cii(Tr, transducer_prop, target_amp):
    trans_amplitude, trans_phase_copmlex = Propagation_C(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_ii(point_amp, target_amp)
      loss += (1/point_num)*(err)
    return loss
    
@jit
def step_Cii(step, opt_state, transducer_prop, target_amp):
  value, grads = jax.value_and_grad(loss_func_Cii)(get_params(opt_state), transducer_prop, target_amp)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
# Main Loop
point_num = 4
x_p = [-0.05, -0.05, 0.05, 0.05]
y_p = [-0.05, 0.05, 0.05, -0.05]
z_p = [0.1, 0.1, 0.1, 0.1]

transducer_prop = prop_matrix(tra_x, tra_y, tra_z, x_p, y_p, z_p, P0, k, r0)

# Initialize tau
target_amp=225.0

In [ ]:
rng = np.random.default_rng(2021) # For reproducibility
amplitude_all = []
phase_all = []
loss_all = []
Tr_A = rng.random((size[0], size[1]))
Tr = [Tr_A]
# Initialize Optimizer
opt_init, opt_update, get_params = jax.experimental.optimizers.adam(0.1, b1=0.9, b2=0.999, eps=1e-08)
opt_state = opt_init(Tr)
loss_list = []
for st in range(dropping_threshold): # Bit messy to list the combination in if-loop, but JAX did not like conditionals passed on. 
  # Making static arguments inputs may improve the situation... 
  value, opt_state = step_Aii(st, opt_state, transducer_prop, target_amp)
  loss_list.append(value)
  
# Exporting Data
Tr = get_params(opt_state)    
part_A = Tr[0]
amplitude = jnp.ones(size)
phase = part_A

amplitude_exports = np.array(amplitude)
phase_exports = np.array(phase)

loss_all.append(np.array(loss_list))
amplitude_all.append(np.array(amplitude_exports.T.ravel()).ravel())
phase_all.append(np.array(phase_exports.T.ravel()).ravel())

phase_1d_arr = np.array( phase_all )
amp_1d_arr = np.array( amplitude_all )
loss_1d_arr = np.array( loss_all )
np.savetxt('results/Loss_exports_N_4_Trans_A_Target_ii_settings.csv', loss_1d_arr, delimiter=',')
np.savetxt('results/Phase_exports_N_4_Trans_A_Target_ii_settings.csv', phase_1d_arr, delimiter=',')
np.savetxt('results/Amplitude_exports_N_4_Trans_A_Target_ii_settings.csv', amp_1d_arr, delimiter=',')

In [ ]:
rng = np.random.default_rng(2021) # For reproducibility
amplitude_all = []
phase_all = []
loss_all = []
Tr_A = rng.random((size[0], size[1]))
Tr_B = rng.random((size[0], size[1]))
Tr = [Tr_A, Tr_B]
# Initialize Optimizer
opt_init, opt_update, get_params = jax.experimental.optimizers.adam(0.1, b1=0.9, b2=0.999, eps=1e-08)
opt_state = opt_init(Tr)
loss_list = []
for st in range(dropping_threshold): # Bit messy to list the combination in if-loop, but JAX did not like conditionals passed on. 
  # Making static arguments inputs may improve the situation... 
  value, opt_state = step_Cii(st, opt_state, transducer_prop, target_amp)
  loss_list.append(value)
  
# Exporting Data
Tr = get_params(opt_state)    
part_A = Tr[0]
part_B = Tr[1]
tr_set = jax.lax.complex(part_A, part_B)
temp_amp = jnp.abs(tr_set);
temp_amp = temp_amp / jnp.max(temp_amp) 
amplitude = temp_amp
phase = jnp.angle(tr_set)

amplitude_exports = np.array(amplitude)
phase_exports = np.array(phase)

loss_all.append(np.array(loss_list))
amplitude_all.append(np.array(amplitude_exports.T.ravel()).ravel())
phase_all.append(np.array(phase_exports.T.ravel()).ravel())

phase_1d_arr = np.array( phase_all )
amp_1d_arr = np.array( amplitude_all )
loss_1d_arr = np.array( loss_all )
np.savetxt('results/Loss_exports_N_4_Trans_C_Target_ii_settings.csv', loss_1d_arr, delimiter=',')
np.savetxt('results/Phase_exports_N_4_Trans_C_Target_ii_settings.csv', phase_1d_arr, delimiter=',')
np.savetxt('results/Amplitude_exports_N_4_Trans_C_Target_ii_settings.csv', amp_1d_arr, delimiter=',')

In [ ]:
!zip -r /content/colab_results.zip /content/results/

updating: content/results/ (stored 0%)
updating: content/results/Phase_exports_N_4_Trans_C_Target_ii_settings.csv (deflated 55%)
updating: content/results/Phase_exports_N_4_Trans_A_Target_ii_settings.csv (deflated 54%)
updating: content/results/Amplitude_exports_N_4_Trans_A_Target_ii_settings.csv (deflated 99%)
updating: content/results/Loss_exports_N_4_Trans_A_Target_ii_settings.csv (deflated 71%)
updating: content/results/Amplitude_exports_N_4_Trans_C_Target_ii_settings.csv (deflated 56%)
updating: content/results/Loss_exports_N_4_Trans_C_Target_ii_settings.csv (deflated 84%)
